#    For the problem -u''+  alpha_omega*u = f_omega
#    B = BilinearForm(fes_state) = grad(u) * grad(v) * dx  + alpha_omega*u*v *dx
based on Newton_Topo_Try2

In [1]:
import numpy as np
import netgen.geom2d as geom2d
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.geom2d import SplineGeometry  # SplieGeometry to define a 2D mesh
from ngsolve.internal import *
from interpolations import InterpolateLevelSetToElems # function which interpolates a levelset function
from scipy.sparse import csr_matrix  # Assuming M.mat and D.mat are CSR sparse matrices
from scipy.sparse.linalg import spsolve
from scipy.sparse.linalg import splu
import time
myMAXH = 0.1
EPS = myMAXH * 1e-6      #variable defining when a point is on the interface and when not

geo = SplineGeometry()

R = 2

## add a rectangle
geo.AddRectangle(p1=(-R,-R),
                 p2=(R,R),
                 bc="rectangle",
                 leftdomain=1,
                 rightdomain=0)
geo.SetMaterial (1, "outer") # give the domain the name "outer"
mesh = Mesh(geo.GenerateMesh(maxh=myMAXH)) # generate ngsolve mesh


fes_state = H1(mesh, order=1)
fes_adj = H1(mesh, order=1)
fes_level = H1(mesh, order=1)

pwc = L2(mesh)   #piecewise constant space

N = fes_state.ndof

## test and trial functions
u, v = fes_state.TnT()

p, q = fes_adj.TnT()

gfu = GridFunction(fes_state)
gfp = GridFunction(fes_adj)

gfud = GridFunction(fes_state)

psi = GridFunction(fes_level)
psi.Set(-1)

psides = GridFunction(fes_level)
psinew = GridFunction(fes_level)

ell=100
print("Degrees of freedom =",N)

Degrees of freedom = 1942


In [2]:
 #Forms And Solver

f1 = 10
f2 = 1
alpha1=7
alpha2=2

f_rhs = GridFunction(pwc)
f_rhs.Set(f1)
alpha = GridFunction(pwc)
alpha.Set(alpha1)

# bilinear form for state 
B = BilinearForm(fes_state)
B += grad(u) * grad(v) * dx
B += alpha*u * v * dx

# bilinear form for adjoint 
B_adj = BilinearForm(fes_adj)
B_adj += grad(p) * grad(q) * dx
B_adj += alpha*p*q*dx


L = LinearForm(fes_state) #Linear form for state
L += f_rhs * v *dx


InterpolateLevelSetToElems(psi, f1, f2, f_rhs, mesh, EPS) # adjust values of RHS according to current PSI
InterpolateLevelSetToElems(psi, alpha1, alpha2, alpha, mesh, EPS) # adjust values of RHS according to current PSI

B.Assemble() #
L.Assemble() # Assemble with the adjusted f_rhs

inv = B.mat.Inverse(inverse="sparsecholesky") # inverse of bilinear form
gfu.vec.data = inv*L.vec
scene_u = Draw(gfu)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

In [3]:

a = 4.0/5.0
b = 2
f = CoefficientFunction( 0.1*( (sqrt((x - a)**2 + b * y**2) - 1) \
                * (sqrt((x + a)**2 + b * y**2) - 1) \
                * (sqrt(b * x**2 + (y - a)**2) - 1) \
                * (sqrt(b * x**2 + (y + a)**2) - 1) - 0.001) )

# solving
psides.Set(f)
InterpolateLevelSetToElems(psides, f1, f2, f_rhs, mesh, EPS)
InterpolateLevelSetToElems(psides, alpha1, alpha2, alpha, mesh, EPS)


B.Assemble()
L.Assemble()
inv = B.mat.Inverse( inverse="sparsecholesky") # inverse of bilinear form

gfud.vec.data = inv*L.vec


Draw(gfud)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene

In [4]:
scene_u = Draw(gfu)
def SolvePDE(adjoint=False):
    #Newton(a, gfu, printing = False, maxerr = 3e-9)

    B.Assemble()
    L.Assemble()

    inv_state = B.mat.Inverse(fes_state.FreeDofs(), inverse="sparsecholesky")

    # solve state equation
    gfu.vec.data = inv_state*L.vec
    scene_u.Redraw()
    if adjoint == True:
        # solve adjoint state equatoin
        duCost.Assemble()
        B_adj.Assemble()
        inv_adj = B_adj.mat.Inverse(fes_adj.FreeDofs(), inverse="sparsecholesky")
        gfp.vec.data = -inv_adj * duCost.vec

InterpolateLevelSetToElems(psi, f1, f2, f_rhs, mesh, EPS)
InterpolateLevelSetToElems(psi, alpha1, alpha2, alpha, mesh, EPS)

#Cost/duCost can only be defined after you solve for u and ud

def Cost(u): 

    return (u - gfud+10*x*y )**2*dx 
    
duCost = LinearForm(fes_adj)
duCost += 2*(gfu-gfud+10*x*y) * q * dx 
#duCost += 0.2*x*y * q *dx
# inout=GridFunction(pwc)
# duCost += inout*ell*dx

# # define the cost function
# def Cost(u):
#     return (u - gfud)**2*dx




SolvePDE(adjoint=True)
#print(gfp.vec)
print("initial cost = ", Integrate(Cost(gfu), mesh))
Draw(gfp)
vector_np = np.array(gfp.vec)

# Count the number of negative elements
num_negative_elements = np.sum(vector_np < 0)

print("Number of negative elements:", num_negative_elements)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

initial cost =  2847.4076688414475


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

Number of negative elements: 1007


In [5]:
DJ=GridFunction(fes_state)
DJ.Set(-(f1-f2)*gfp)
Draw(DJ)
#This is the generalized topological derivative

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene

In [5]:
N = fes_state.ndof
phi1,phi2 = fes_state.TnT()
M = BilinearForm(fes_state)
M += 2*phi1*phi2*dx
# D = BilinearForm(fes_state)
# D += grad(phi1) * grad(phi2) * dx
# D += phi1*phi2*dx

M.Assemble()
#D.Assemble()

M_CSR = M.mat.CSR()
#D_CSR = D.mat.CSR()

M_dense = csr_matrix((M_CSR[0],M_CSR[1],M_CSR[2]),shape=(N, N)).toarray()
#D_dense =  csr_matrix((D_CSR[0],D_CSR[1],D_CSR[2]),shape=(N, N)).toarray()

X = ( (alpha1-alpha2)-(f1-f2) )* np.eye(N)
X1 =( (alpha1-alpha2)-(f1-f2) )* np.eye(N)
zer = np.zeros((N,N))
zer1 = np.zeros((N,N))
Y = np.zeros((N, N))
Y1 = np.zeros((N, N))


# Print the shape of the block matrix
#print("Block matrix shape:", block_matrix.shape)

# number of non-zero elements in the block matrix
# nonzeros = np.count_nonzero(block_matrix)
# print("Number of nonzeros in the block matrix:", nonzeros)
# print("Degrees of freedom:",N)

In [6]:
iter_max = 50
converged = False
direction = GridFunction(fes_state)

nonoptimalpnts = []
itera =[]
N = fes_state.ndof

kappa = 1
kappa_min = 0.01
#psi.Set(-1)
xm=0.
ym=0.
psi.Set( (x-xm)**2+(y-ym)**2-0.25**2)
psinew.vec.data= psi.vec
scene_u = Draw(gfu) ##DRAW
scene_psi = Draw(IfPos(psi,1,-1),mesh)
J = Integrate(Cost(gfu),mesh)
print("initial Cost",J)
with TaskManager():

    for k in range(iter_max):
        print("================ iteration ", k, "===================")

        # copy new levelset data from psinew into psi
        psi.vec.data = psinew.vec
        scene_psi.Redraw()
        time.sleep(7)
        InterpolateLevelSetToElems(psi, f1, f2, f_rhs, mesh, EPS)
        InterpolateLevelSetToElems(psi, alpha1, alpha2, alpha, mesh, EPS) # adjust values of RHS according to current PSI


        SolvePDE(adjoint=True)
        scene_u.Redraw()
        vector_np = np.array(gfp.vec)

        # Count the number of negative elements


        J_current = Integrate(Cost(gfu),mesh)

        print("cost in beginning of iteration", k, ": Cost = ", J_current)
        
        # RHS
        BB = np.zeros(3 * N) # Initializing RHS
        indices = []  
        for i in range(N):
            BB[i] = ((alpha1-alpha2)-(f1 - f2)) * gfp.vec[i]  #Calculating RHS and saving indices of "optimal" points 
            if psi.vec[i] < 0 and BB[i] > 0:
                indices.append(i)
            elif psi.vec[i] > 0 and BB[i] < 0:
                indices.append(i)
        nonoptimalpnts.append(len(indices))
        itera.append(k)
        for i in range(N):
            if i not in indices:
                BB[i]=0
        for i in indices: # Editing the matrices according to the indices 
            zer1[i,i]=1
            Y[:,i]=0
            X[:,i]=0
            Y[i,:]=0
            X[i,:]=0
            
        print("len(indices)",len(indices))
        if len(indices)<int(0.005*N):
            print("99.5% of elements are optimum")
            break

        D = BilinearForm(fes_state)
        D += grad(phi1) * grad(phi2) * dx
        D += alpha*phi1*phi2*dx
        D.Assemble()
        D_CSR = D.mat.CSR()
        D_dense =  csr_matrix((D_CSR[0],D_CSR[1],D_CSR[2]),shape=(N, N)).toarray()

        block_matrix = np.block([ #Constructing 3N system
            [zer1, Y, X],
            [Y.T, M_dense, D_dense],
            [X.T, D_dense.T, zer]
        ])

        #Solving 3N system
        A=csr_matrix(block_matrix)
        XX = spsolve(A, BB )     
        delta_x = XX[:N]
        direction.vec.data=delta_x
        
        # norm_direction = sqrt(Integrate(direction**2*dx, mesh)) # L2 norm of TD_node
        # direction.vec.data = 1/norm_direction * direction.vec # normalised TD_node

        # normPsi = sqrt(Integrate(psi**2*dx, mesh)) # L2 norm of psi
        # psi.vec.data = 1/normPsi * psi.vec  # normalised psi
        for j in range(1):

            # update the level set function
            psinew.vec.data = psi.vec + 1*direction.vec

            # update beta and f_rhs
            InterpolateLevelSetToElems(psinew, f1, f2, f_rhs, mesh, EPS)
            InterpolateLevelSetToElems(psinew, alpha1, alpha2, alpha, mesh, EPS)

            # solve PDE without adjoint
            SolvePDE()

            Redraw(blocking=True)

            Jnew = Integrate(Cost(gfu), mesh)

            if Jnew > J_current:
                print("--------------------")
                print("-----line search ---")
                print("--------------------")
                kappa = kappa*0.8
                kappa = max(kappa_min,kappa)
                if kappa == kappa_min:
                    break
                print("kappa", kappa)
            else:
                kappa = 1.2*kappa
                break

        Redraw(blocking=True)
        kappa = min(1, kappa*1.2)

        print("----------- Jnew in  iteration ", k, " = ", Jnew, " (kappa = ", kappa, ")")
        print('')
        print("iter" + str(k) + ", Jnew = " + str(Jnew) + " (kappa = ", kappa, ")")

        print("end of iter " + str(k))



WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

initial Cost 2847.4076688414475
================ iteration  0 ===================
cost in beginning of iteration 0 : Cost =  2849.0890993984863
len(indices) 1034
----------- Jnew in  iteration  0  =  2741.5021339308896  (kappa =  1 )

iter0, Jnew = 2741.5021339308896 (kappa =  1 )
end of iter 0
================ iteration  1 ===================
cost in beginning of iteration 1 : Cost =  2741.5021339308896
len(indices) 276
----------- Jnew in  iteration  1  =  2737.852372576701  (kappa =  1 )

iter1, Jnew = 2737.852372576701 (kappa =  1 )
end of iter 1
================ iteration  2 ===================
cost in beginning of iteration 2 : Cost =  2737.852372576701
len(indices) 168
----------- Jnew in  iteration  2  =  2736.828978457067  (kappa =  1 )

iter2, Jnew = 2736.828978457067 (kappa =  1 )
end of iter 2
================ iteration  3 ===================
cost in beginning of iteration 3 : Cost =  2736.828978457067
len(indices) 115
----------- Jnew in  iteration  3  =  2736.395563160986

In [ ]:
# #Solve by Spsolve
# # Construct the block matrix


# A=csr_matrix(block_matrix)

# XX = spsolve(A, BB )


# delta_x = XX[:N]
# direction = GridFunction(fes_state)
# direction.vec.data = delta_x
# Draw(direction,mesh)
# print(delta_x)

In [ ]:
# for i in indices:
#     print("delta_x[",i,"]",BB[i])


for i in range(500,515):
    print("delta_x[",i,"]",BB[i])

In [ ]:
print(len(indices))


After 25 steps, starting with kappa = 1 and line search,
as the DJ optimum is closer to zero (small shift magnitude) 

For the shift 10xy, the number of indices is 15 out of order (-2) while all other 1943 elements are positive of order (150) 
For the shift xy, the number of indices is 658 out of order (-1) while most of the other 1943 elements are positive of order (10)

In [ ]:
import csv

# Save data to CSV
with open('Non_opt_pnts.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['Iterations', 'Non-Optimal Points'])
    for i in range(len(itera)):
        csvwriter.writerow([itera[i], nonoptimalpnts[i]])

print("Data saved to data.csv")